# Report04 - Nathan Yee

This notebook contains report04 for computational baysian statistics fall 2016

MIT License: https://opensource.org/licenses/MIT

In [1]:
from __future__ import print_function, division

% matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np

from thinkbayes2 import Pmf, Cdf, Suite, Joint
import thinkplot

## Parking meter theft

From DASL(http://lib.stat.cmu.edu/DASL/Datafiles/brinkdat.html)

>The variable CON in the datafile Parking Meter Theft represents monthly parking meter collections by the principle contractor in New York City from May 1977 to March 1981. In addition to contractor collections, the city made collections from a number of "control" meters close to City Hall. These are recorded under the varia- ble CITY. From May 1978 to April 1980 the contractor was Brink's. In 1983 the city presented evidence in court that Brink's employees has been stealing parking meter moneys - delivering to the city less than the total collections. The court was satisfied that theft has taken place, but the actual amount of shortage was in question. Assume that there was no theft before or after Brink's tenure and estimate the monthly short- age and its 95% confidence limits.

So we are asking three questions. What is the probability that that money has been stolen? And how much money was stolen?

This problem is similar to that of "Improving Reading Ability" by Allen Downey

To do this, we want to calculate  First we load our data from the csv file.

In [2]:
import pandas as pd

df = pd.read_csv('parking.csv', skiprows=17, delimiter='\t')
df.head()

,TIME,CON,CITY,BRINK
0,1,2224277,6729,0
1,2,1892672,5751,0
2,3,1468074,6711,0
3,4,1618966,7069,0
4,5,1509195,7134,0


First we need to normalize the CON (contractor) collections by the amount gathered by the CITY. This will give us a ratio of contractor collections to city collections. If we just use the raw contractor collections, fluctuations throughout the months could mislead us.

In [3]:
df['RATIO'] = df['CON'] / df['CITY']

Next, lets see what the means of the RATIO data compare between the general contractors and BRINK.

In [4]:
grouped = df.groupby('BRINK')
for name, group in grouped:
    print(name, group.RATIO.mean())

0 244.681143201
1 229.583858011


We see that for a dollar gathered by the city, general contractors report 244.7 dollars while BRINK only reports 230 dollars.

Now, we will fit the data to a Normal class to compute the likelihood of a sameple from the normal distribution. This is a similar process to what we did in the improved reading ability problem.

In [5]:
from scipy.stats import norm

class Normal(Suite, Joint):
    
    def Likelihood(self, data, hypo):
        """
        
        data: sequence of test scores
        hypo: mu, sigma
        """
        mu, sigma = hypo
        likes = norm.pdf(data, mu, sigma)
        return np.prod(likes)

# How good is that cereal?

In [57]:
df = pd.read_csv('cereal.csv', skiprows=0, delimiter='\t')
df = df[['name','calories', 'protein', 'fat', 'sodium', 'fiber', 'carbo', 'cups', 'rating']]
df.head()

,name,calories,protein,fat,sodium,fiber,carbo,cups,rating
0,100%_Bran,70,4,1,130,10,5,0.33,68.402973
1,100%_Natural_Bran,120,3,5,15,2,8,1.00,33.983679
2,All-Bran,70,4,1,260,9,7,0.33,59.425505
3,All-Bran_with_Extra_Fiber,50,4,0,140,14,8,0.50,93.704912
4,Almond_Delight,110,2,2,200,1,14,0.75,34.384843


In [55]:
with_cups = df[['calories','protein','fat','sodium','fiber','carbo']].div(df.cups, axis=0)

In [74]:
df1 = df.concat([df['name'], with_cups], axis=1)
df1.head()

AttributeError: 'DataFrame' object has no attribute 'concat'